In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
cal_len = 5

In [3]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_index = {idx: char for idx, char in enumerate(chars)}

In [4]:
# 학습 데이터 생성
train_data = np.random.choice(list(chars), size = (10000, cal_len))
label_data = np.flip(train_data, axis = 1)

In [5]:
# 문자 데이터를 숫자 인덱스로 변환
train_data_indices = np.array([[char_to_index[char] for char in seq] for seq in train_data])
label_data_indices = np.array([[char_to_index[char] for char in seq] for seq in label_data])

print(train_data[:4])
print(label_data[:4])

[['D' 'W' 'n' 'T' 'D']
 ['f' 'a' 'P' 'n' 'g']
 ['Y' 'G' 'i' 'D' 'd']
 ['u' 'K' 'x' 'z' 'i']]
[['D' 'T' 'n' 'W' 'D']
 ['g' 'n' 'P' 'a' 'f']
 ['d' 'D' 'i' 'G' 'Y']
 ['i' 'z' 'x' 'K' 'u']]


In [6]:
# pad_sequence 전처리
padded_train = pad_sequences(train_data_indices, maxlen = cal_len)
padded_labels = pad_sequences(label_data_indices, maxlen = cal_len)

In [7]:
# 모델링
model = Sequential()
model.add(Embedding(num_classes, 100, input_length = cal_len))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dense(num_classes, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            5200      
                                                                 
 bidirectional (Bidirectiona  (None, 5, 128)           84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 5, 52)             6708      
                                                                 
Total params: 96,388
Trainable params: 96,388
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 모델 컴파일
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [9]:
# 모델 실행
history = model.fit(padded_train, padded_labels, epochs = 50, batch_size = 32, validation_split = 0.2)

Epoch 1/50
250/250 [==============================] - 3s 8ms/step - loss: 3.0178 - accuracy: 0.2713 - val_loss: 1.8715 - val_accuracy: 0.4328
Epoch 2/50
250/250 [==============================] - 2s 6ms/step - loss: 1.5908 - accuracy: 0.4651 - val_loss: 1.4549 - val_accuracy: 0.4745
Epoch 3/50
250/250 [==============================] - 2s 7ms/step - loss: 1.3323 - accuracy: 0.5018 - val_loss: 1.2913 - val_accuracy: 0.5034
Epoch 4/50
250/250 [==============================] - 2s 7ms/step - loss: 1.2049 - accuracy: 0.5330 - val_loss: 1.2132 - val_accuracy: 0.5249
Epoch 5/50
250/250 [==============================] - 2s 7ms/step - loss: 1.1141 - accuracy: 0.5623 - val_loss: 1.1454 - val_accuracy: 0.5414
Epoch 6/50
250/250 [==============================] - 2s 7ms/step - loss: 1.0396 - accuracy: 0.5885 - val_loss: 1.0928 - val_accuracy: 0.5686
Epoch 7/50
250/250 [==============================] - 2s 7ms/step - loss: 0.9744 - accuracy: 0.6128 - val_loss: 1.0296 - val_accuracy: 0.5867
Epoch 

In [10]:
# loss와 accuracy
loss, acc = model.evaluate(padded_train, padded_labels)
print(f"loss : {loss: .4f}, acc = {acc: .4f}")

313/313 [==============================] - 1s 4ms/step - loss: 0.0392 - accuracy: 0.9925
loss :  0.0392, acc =  0.9925


In [11]:
# 랜덤한 영문 대소문자 5글자를 요소로 하는 20개 리스트 생성
test_data = ["".join(np.random.choice(list(chars), cal_len)) for _ in range(20)]

In [12]:
# 거꾸로된 리스트 생성
expected_outputs = [s[::-1] for s in test_data]

In [13]:
correct_predictions = 0
total_predictons = len(test_data)

for i, test_string in enumerate(test_data):
    numbers = [char_to_index[char] for char in test_string]
    array_numbers = np.array([numbers])
    padded_numbers = pad_sequences(array_numbers, maxlen = cal_len)
    prediction = model.predict(padded_numbers)
    predicted_indices = np.argmax(prediction, axis = -1)[0]
    predicted_chars = [index_to_index[idx] for idx in predicted_indices]
    predicted_output = "".join(predicted_chars)

    is_correct = predicted_output == expected_outputs[i]
    if is_correct:
        correct_predictions += 1
    print(f"입력: {test_string}")
    print(f"예측된 출력:  {predicted_output}")
    print(f"실제 출력: {expected_outputs[i]}")
    print(f"정확 여부: {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictions / total_predictons
print(f"총 정확도:  {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 357ms/step
입력: mYUCK
예측된 출력:  KCUYm
실제 출력: KCUYm
정확 여부: 맞음

1/1 [==============================] - 0s 13ms/step
입력: mhcmw
예측된 출력:  wmcmm
실제 출력: wmchm
정확 여부: 틀림

1/1 [==============================] - 0s 12ms/step
입력: cjCNv
예측된 출력:  vNCjc
실제 출력: vNCjc
정확 여부: 맞음

1/1 [==============================] - 0s 13ms/step
입력: cLemH
예측된 출력:  HmeLc
실제 출력: HmeLc
정확 여부: 맞음

1/1 [==============================] - 0s 13ms/step
입력: aMfip
예측된 출력:  pifMa
실제 출력: pifMa
정확 여부: 맞음

1/1 [==============================] - 0s 16ms/step
입력: lruHC
예측된 출력:  CHuul
실제 출력: CHurl
정확 여부: 틀림

1/1 [==============================] - 0s 17ms/step
입력: Qmoim
예측된 출력:  mmomQ
실제 출력: miomQ
정확 여부: 틀림

1/1 [==============================] - 0s 13ms/step
입력: vUauh
예측된 출력:  huaUv
실제 출력: huaUv
정확 여부: 맞음

1/1 [==============================] - 0s 13ms/step
입력: QtbhD
예측된 출력:  DhbtQ
실제 출력: DhbtQ
정확 여부: 맞음

1/1 [==============================] - 0s 14ms/step
입력: Rgopo
예측된 출력:  ooogR
실제 출력: opogR
